In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm

## Hyperparams

In [3]:
n_epochs = 100
batch_size = 256
n_features = 128
projection_dim = 128
weight_decay = 1e-4
lr = 1e-3

n_channels = 5
seq_len = 200
n_classes = 10

vis_freq = 10

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cpu


In [2]:
data_dir = "./data/"
# O1 O2 T5 T6
# bandpass 5-95 Hz

## Load Data

In [5]:
import datasets

d = datasets.load_from_disk(data_dir + "dataset_train_preprocessed")

/Users/sun/miniconda3/envs/cmput328/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


FileNotFoundError: Directory ./data/dataset_train_preprocessed not found

In [ ]:
train_eegs = torch.tensor(d["pixel_values"])
train_labels = torch.tensor(d["label"])

In [ ]:
print(train_eegs.shape, train_eegs.dtype)
print(train_labels.shape, train_labels.dtype)

torch.Size([10436, 256, 5]) torch.float32
torch.Size([10436]) torch.int64


In [ ]:
from torch.utils.data import Dataset

class EEGDataset(Dataset):
    def __init__(self, eegs, labels, transform=None):
        self.eegs = eegs
        self.labels = labels
        self.transform = transform

    def __getitem__(self, idx):
        eeg = self.eegs[idx]
        label = self.labels[idx]

        if self.transform:
            eeg = self.transform(eeg)

        return eeg, label

    def __len__(self):
        return len(self.eegs)

train_data = EEGDataset(train_eegs, train_labels)

In [ ]:
eeg, label = train_data[0]
print(eeg.shape, label)

torch.Size([256, 5]) tensor(6)


## Define Network

In [32]:
import torch
from typing import Any, Optional, Tuple, Union
import torch.nn as nn
import torch.nn.functional as F

class EEGFeatNet(nn.Module):
    def __init__(self, n_channels, n_features, projection_dim, num_layers=1):
        super(EEGFeatNet, self).__init__()
        self.hidden_size = n_features
        self.num_layers = num_layers

        self.encoder = nn.LSTM(input_size=n_channels, hidden_size=self.hidden_size, num_layers=self.num_layers, batch_first=True)
        self.fc = nn.Linear(in_features=self.hidden_size, out_features=projection_dim, bias=False)


    def forward(
        self,
        pixel_values: Optional[torch.FloatTensor] = None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
    ):

        device = next(self.parameters()).device
        x = pixel_values
        h_n = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c_n = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)

        _, (h_n, c_n) = self.encoder(x, (h_n, c_n))
        x = h_n[-1]

        x = F.normalize(x, dim=-1)

        return x


In [33]:
device = "mps"
eeg = torch.randn((batch_size, seq_len, n_channels)).to(device)
model = EEGFeatNet(n_channels=n_channels, n_features=n_features, projection_dim=projection_dim).to(device)

proj = model(eeg)
print(proj.shape)

torch.Size([256, 128])


In [10]:
f"All Parameters: {sum(p.numel() for p in model.parameters())}"

'All Parameters: 85504'

## Dataloaders

In [5]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)

NameError: name 'train_data' is not defined

In [ ]:
for eeg, label in train_loader:
    eeg = eeg.to(device)
    proj = model(eeg)
    print(proj.shape)

torch.Size([256, 128])
torch.Size([256, 128])
torch.Size([256, 128])
torch.Size([256, 128])
torch.Size([256, 128])
torch.Size([256, 128])
torch.Size([256, 128])
torch.Size([256, 128])
torch.Size([256, 128])
torch.Size([256, 128])
torch.Size([256, 128])
torch.Size([256, 128])
torch.Size([256, 128])
torch.Size([256, 128])
torch.Size([256, 128])
torch.Size([256, 128])
torch.Size([256, 128])
torch.Size([256, 128])
torch.Size([256, 128])
torch.Size([256, 128])
torch.Size([256, 128])
torch.Size([256, 128])
torch.Size([256, 128])
torch.Size([256, 128])
torch.Size([256, 128])
torch.Size([256, 128])
torch.Size([256, 128])
torch.Size([256, 128])
torch.Size([256, 128])
torch.Size([256, 128])
torch.Size([256, 128])
torch.Size([256, 128])
torch.Size([256, 128])
torch.Size([256, 128])
torch.Size([256, 128])
torch.Size([256, 128])
torch.Size([256, 128])
torch.Size([256, 128])
torch.Size([256, 128])
torch.Size([256, 128])
torch.Size([196, 128])


## Train Loop

In [ ]:
from visualization import K_means

def train(epoch, model, optimizer, loss_fn, miner, train_dataloader):
    tq = tqdm(train_dataloader)
    for batch_idx, (eeg, label) in enumerate(tq, start=1):
        eeg    = eeg.to(device)
        label = label.to(device)

        optimizer.zero_grad()

        x_proj = model(eeg)

        hard_pairs = miner(x_proj, labels)
        loss = loss_fn(x_proj, labels, hard_pairs)
        
        loss.backward()
        optimizer.step()

        running_loss = running_loss + [loss.detach().cpu().numpy()]
                
        tq.set_description('Train:[{}, {:0.3f}]'.format(epoch, loss.item()))

    if (epoch % vis_freq) == 0:
        for batch_idx, (eeg, labels) in enumerate(tqdm(train_dataloader)):
            eeg, labels = eeg.to(device), labels.to(device)
            with torch.no_grad():
                x_proj = model(eeg)
            
            eeg_featvec_proj = np.concatenate((eeg_featvec_proj, x_proj.cpu().detach().numpy()), axis=0) if eeg_featvec_proj.size else x_proj.cpu().detach().numpy()
            
            labels_array     = np.concatenate((labels_array, labels.cpu().detach().numpy()), axis=0) if labels_array.size else labels.cpu().detach().numpy()

        ### compute k-means score and Umap score on the text and image embeddings
        k_means = K_means(n_clusters=n_classes)
        clustering_acc_proj = k_means.transform(eeg_featvec_proj, labels_array)
        print("[Epoch: {}, Train KMeans score Proj: {}]".format(epoch, clustering_acc_proj))

    return running_loss

In [ ]:
from pytorch_metric_learning import miners, losses, distances

model = EEGFeatNet(n_channels=n_channels, n_features=n_features, projection_dim=projection_dim).to(device)
optimizer = torch.optim.AdamW(list(model.parameters()), lr=lr, weight_decay=weight_decay)

# miner = miners.MultiSimilarityMiner()
# loss_fn = losses.TripletMarginLoss()
margin = 0.2
distance = distances.LpDistance()
loss_fn = losses.TripletMarginLoss(margin, distance=distance)
miner = miners.TripletMarginMiner(margin, "semihard", distance=distance)

In [ ]:
if device == "cuda":
    torch.cuda.empty_cache()

In [ ]:
for epoch in range(n_epochs):
    running_train_loss = train(epoch, model, optimizer, loss_fn, miner, train_loader)

100%|██████████| 41/41 [00:00<00:00, 114.46it/s]


[Epoch: 0, Train KMeans score Proj: 0.112878497508624]


100%|██████████| 41/41 [00:00<00:00, 113.60it/s]


[Epoch: 10, Train KMeans score Proj: 0.11144116519739364]


100%|██████████| 41/41 [00:00<00:00, 117.05it/s]


...

100%|██████████| 41/41 [00:00<00:00, 117.74it/s]
c:\Users\ruoto\miniconda3\envs\cmput328\lib\site-packages\sklearn\base.py:1152: ConvergenceWarning: Number of distinct clusters (8) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)


[Epoch: 80, Train KMeans score Proj: 0.10367957071674971]


100%|██████████| 41/41 [00:00<00:00, 115.83it/s]
c:\Users\ruoto\miniconda3\envs\cmput328\lib\site-packages\sklearn\base.py:1152: ConvergenceWarning: Number of distinct clusters (8) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)


[Epoch: 90, Train KMeans score Proj: 0.10367957071674971]


Train:[99, 0.001]: 100%|██████████| 41/41 [00:00<00:00, 49.92it/s]
